# 基于TextCNN的文本分类

### 引入必要的包
本教程采用Python3.5+、Tensorflow1.8+ 进行讲解

Tensorflow是Google开源的并行神经网络计算框架，可以利用CPU,GPU,TPU资源对神经网络进行并行计算，底层采用C++实现

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
print(tf.__version__)

1.8.0


### 定义建模需要的全局变量

In [29]:
# 文本序列的长度
sequence_length = 200
# 分类个数
num_classes = 10

# 定义embedding_size
embedding_size = 200

# 定义词表大小
vocab_size = 50000

# 不同卷积层卷积核的定义
filter_sizes = [2, 3, 4, 5]

# 输出特征图数目
num_filters = 16

# dropout 保留比例
dropout_keep_prob = 0.7

# 正则化
l2_reg_lambda = 0.01

### 定义输入输出变量

通常输入是指输入到神经网络建模的变量x
输出一般是预测的变量或者标签y

**placeholder:** placeholder 是 Tensorflow 中的占位符，暂时储存变量，可以定义输入张量的类型，输入张量的形状

**variable:**  variable可以是一个初始化的变量

In [35]:
# 定义输入的变量x：表示输入一条长度为sequence_length的句子或文章
x = tf.placeholder(dtype=tf.int64, shape=[None, sequence_length], name="input_x")
# 定义输出的变量y: 表示分类输出的概率
y = tf.placeholder(dtype=tf.int64, shape=[None, num_classes])

### Embedding层定义
Embedding：简单来说，就是应对高维、稀疏的id类特征，通过将单个id（可能是某个词的id，也可能是某个商品的id）映射成一个稠密向量，变id特征的“精确匹配”为embedding向量的“模糊查找”，从而提升算法的扩展能力。

举个例子：如果要表示一组文本序列单词所在的位置 \["我", "是", "中国人", "。"\]该怎么表示呢？

方法1： \[3, 2, 4, 1\],分别表示词语“我”、“是”、“中国人”和句号所处的位置分别为3， 2， 4， 1 

方法2：\[[0, 0, 1, 0], 
[0, 1, 0, 0], 
[0, 0, 0, 1], 
[1, 0, 0, 0]\] 

其中两种表示都是表示词语出现的位置，其中方法一是该位置的稠密表示，方法二为稀疏表示

tf.embedding_lookup:
所谓embedding_lookup(W, id1)，可以想像成一个只在id1位为1的\[1, vocab_size\]的one_hot向量，与\[vocab_size, embed_size\]的W矩阵相乘，结果是一个\[1, embed_size\]的向量，它就是id1对应的embedding向量，实际上就是W矩阵的第id1行。

![](./images/lookup.jpg)

### 对输入的文本序列进行embedding_lookup训练

In [5]:
# 定义embedding_lookup 所需要的权重矩阵W
W = tf.Variable(tf.random_uniform(shape=[vocab_size, embedding_size]), name="embedding_w")

In [6]:
# 对输入的文本序列进行embedding_lookup训练
embedding_layer = tf.nn.embedding_lookup(W, x)
# 注意此时embedding_layer的输出维度为 [batch_size, vocab_size, embedding_size] 如果要进行conv2d卷积必须输入4维的张量
embedding_layer = tf.expand_dims(embedding_layer, -1)

### 卷积层的定义

- **常规卷积层工作原理：**

![](./images/conv_net.gif)

- **TextCNN 卷积神经网络原理：**

![](./images/textcnn.png)


- **卷积层需定义的参数:**

**卷积核：** Tensorflow API要求输入的卷积核大小为 \[**高度**, **宽度**, **输入的通道数**, **输出的通道数**\]

**步长定义：** strides定义为 一组int型数值表示卷积核在输入的4维张量在每个维度上滑动的步幅大小

**填充方式** padding定义为"SAME","VALID"两种方式

- **池化层需定义的参数:**

**ksize：**  输入张量在每个维度上的窗口大小

**步长定义：** strides定义为 一组int型数值表示卷积核在输入的4维张量在每个维度上滑动的步幅大小

**填充方式** padding定义为"SAME","VALID"两种方式


In [7]:
tf.nn.max_pool??

In [8]:
output_layers = []
for i, filter_size in enumerate(filter_sizes):
    filter_shape = [filter_size, embedding_size, 1, num_filters]
    w = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=0.1))
    conv = tf.nn.conv2d(
        input=embedding_layer,
        filter=,
        strides=[1, 1, 1, 1],
        padding='VALID'
    )
    # 疑问？？？ 卷积层输出维度是多少呢？
    #
    # Bias定义
    bias = tf.constant(0.5, [num_filters])
    # activation_function(w*x+b)
    # 对卷积层进行激活
    conv_layer = tf.nn.relu(tf.nn.bias_add(conv, bias))
    # 对卷积层输出进行最大值池化
    pool_layer = tf.nn.max_pool(
        value=conv_layer,
        ksize=[1, sequence_length - filter_size + 1, 1, 1],
        strides=[1, 1, 1, 1],
        padding="VALID"
    )
    output_layers.append(pool_layer)

In [13]:
# 将池化后的卷积层拼接起来展开成一个稠密的向量
num_filters_total = num_filters*len(filter_sizes)
pool_flatten = tf.reshape(output_layers, [-1, ])

In [10]:
# 对展开的稠密向量做dropout
dropout_layer = tf.nn.dropout(pool_flatten, keep_prob=dropout_keep_prob)

### 输出层的定义

输出层可以将卷积后池化层的拼接向量进行线性分类激活

即对输出层进行建模：soft_max(W*x + b）

In [11]:
dropout_layer.shape

TensorShape([Dimension(None), Dimension(64)])

In [17]:
W = tf.get_variable(
    name="W",
    shape=[num_filters_total, num_classes],
    initializer=tf.contrib.layers.xavier_initializer()
)

In [18]:
b = tf.Variable(tf.constant(0.1, shape=[num_classes]))

In [20]:
l2_loss = tf.constant(0.0)
l2_loss += tf.nn.l2_loss(W)
l2_loss += tf.nn.l2_loss(b)

In [27]:
logits = tf.nn.xw_plus_b(dropout_layer, W, b)

### 定义损失函数评估输出层的损失大小

In [28]:
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)

In [30]:
loss = tf.reduce_mean(entropy) + l2_reg_lambda * l2_loss

### 对损失进行优化

In [31]:
# 定义优化器，目标是最小化loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-4).minimize(loss)
# 输入预处理的数据进行迭代优化

### 性能评估

In [36]:
correct_prediction = tf.equal(tf.argmax(logits, axis=1), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))